In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")

In [30]:
from springstone.data import get_data, get_missing_dates, create_train_test, create_df_for_prophet, download_model
from springstone.utils import bollinger_bands, moving_average, prophet_preprocessing, prophet_non_business_days
from springstone.utils import basic_recommendation, temp_data_predict, next_business_day
from springstone.params import MODEL_TYPE
import pandas as pd
import numpy as np
from prophet import Prophet
from prophet.plot import plot_forecast_component_plotly
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import joblib
from termcolor import colored
from datetime import date, timedelta
import holidays

In [3]:
ticker = 'AAPL'

In [4]:
hist = get_data(ticker)

[*********************100%***********************]  1 of 1 completed


In [5]:
hist

,Open,High,Low,Close,Volume
Date,,,,,
2012-02-21,18.102858,18.387501,18.004286,18.387501,605595200
2012-02-22,18.324286,18.410357,18.181070,18.322857,483302400
2012-02-23,18.395714,18.493929,18.196428,18.442499,568027600
2012-02-24,18.559643,18.674999,18.522858,18.657499,415072000
2012-02-27,18.618214,18.875000,18.438572,18.777143,547582000
...,...,...,...,...,...
2022-02-14,167.369995,169.580002,166.559998,168.880005,86185500
2022-02-15,170.970001,172.949997,170.250000,172.789993,62527400
2022-02-16,171.850006,173.339996,170.050003,172.550003,61177400


In [6]:
df_prophet = prophet_preprocessing(hist, 'Close')

In [7]:
non_business_days_df = prophet_non_business_days(hist)

In [8]:
df_train, df_test = create_train_test(df_prophet)

In [9]:
df_train

,ds,y
0,2012-02-21,18.387501
1,2012-02-22,18.322857
2,2012-02-23,18.442499
3,2012-02-24,18.657499
4,2012-02-25,18.657499
...,...,...
2916,2020-02-15,81.237503
2917,2020-02-16,81.237503
2918,2020-02-17,81.237503
2919,2020-02-18,79.750000


In [10]:
df_test

,ds,y
2921,2020-02-20,80.074997
2922,2020-02-21,78.262497
2923,2020-02-22,78.262497
2924,2020-02-23,78.262497
2925,2020-02-24,74.544998
...,...,...
3646,2022-02-14,168.880005
3647,2022-02-15,172.789993
3648,2022-02-16,172.550003
3649,2022-02-17,168.880005


In [11]:
m_prophet = Prophet(holidays=non_business_days_df)
m_prophet.fit(df_train)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Initial log joint probability = -80.3622


    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
      99       7267.21     0.0517129       1160.68       1.166      0.1166      121   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     199       7303.37     0.0106924       456.687       2.431     0.02431      237   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     299       7311.95     0.0102775       774.401           1           1      352   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     399       7319.48    0.00887635       625.283      0.5338           1      477   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     499       7322.74   0.000563668       90.4905           1           1      602   
    Iter      log prob        ||dx||      ||grad||       alpha      alpha0  # evals  Notes 
     538     

In [12]:
joblib.dump(m_prophet, 'model_prophet_AAPL.joblib')

['model_prophet_AAPL.joblib']

In [33]:
basic_recommendation(ticker)

[*********************100%***********************]  1 of 1 completed


'Buy recommendation'

In [34]:
next_business_day(date.today())

datetime.date(2022, 3, 2)

In [ ]:
n_days_prediction=1
future = pipeline.make_future_dataframe(periods=n_days_prediction)
future

In [ ]:
forecast = pipeline.predict(future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'non business day']]

In [ ]:
m_prophet.plot(forecast);

In [ ]:
m_prophet.plot_components(forecast);

In [ ]:
#plot_forecast_component_plotly(m_prophet, forecast, 'non business day')

In [ ]:
forecast = forecast[- n_days_prediction:][['ds', 'yhat', 'yhat_lower', 'yhat_upper', 'non business day']]

In [ ]:
forecast

In [ ]:
df_test[:n_days_prediction]

In [ ]:
mae_prophet = mean_absolute_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
mape_prophet = mean_absolute_percentage_error(df_test[:n_days_prediction]['y'], forecast['yhat'])
print(f'MAE: {mae_prophet}')
pbrint(f'MAPE: {mape_prophet}')